In [1]:
import nltk
import numpy as np
from collections import Counter

In [2]:
np.random.seed(15)

Готовим данные:

In [3]:
corpus = ''
for filename in ['AnnaKarenina.txt', 'WarAndPeace.txt']:
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            corpus += ' ' + line

In [4]:
RUSSIAN_SYMBOLS = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

def filter_extra_symbols(text):
    res = ''
    for symbol in text:
        if symbol in RUSSIAN_SYMBOLS:
            res += symbol
    return res

def preprocess_text(text):
    text_in_lowercase = text.lower()
    filtered_text = filter_extra_symbols(text_in_lowercase)
    return ' '.join(nltk.tokenize.RegexpTokenizer(r"\w+").tokenize(filtered_text))

In [5]:
corpus = preprocess_text(corpus)

In [6]:
corpus[:100]

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все'

### Вспомогательные функции

In [7]:
def get_ngram_frequencies(text, n=1):
    if n == 1:
        freqs = dict()
        for symbol, count in Counter(text).items():
            freqs[symbol] = count / len(text)
        return freqs
    ngrams = nltk.everygrams(text, min_len=n, max_len=n)
    frequencies = nltk.FreqDist(ngrams)
    count = 0
    for _, c in frequencies.items():
        count += c
    frequencies = {
        ''.join(ngram): frequency / count for ngram, frequency in frequencies.items()
    }
    return frequencies

In [8]:
get_ngram_frequencies('аббввв', n=1)

{'а': 0.16666666666666666, 'б': 0.3333333333333333, 'в': 0.5}

In [9]:
get_ngram_frequencies('аббввв', n=2)

{'аб': 0.2, 'бб': 0.2, 'бв': 0.2, 'вв': 0.4}

In [10]:
def get_random_mapping():
    new_symbols = np.random.choice(RUSSIAN_SYMBOLS,
                                   size=len(RUSSIAN_SYMBOLS),
                                   replace=False)
    mapping = dict()
    for idx in range(len(RUSSIAN_SYMBOLS)):
        mapping[RUSSIAN_SYMBOLS[idx]] = new_symbols[idx]
    return mapping

In [11]:
test_frequencies = get_ngram_frequencies('аббввв', n=1)
get_random_mapping()

{'а': 'ш',
 'б': 'н',
 'в': 'я',
 'г': 'п',
 'д': 'ч',
 'е': 'у',
 'ё': 'т',
 'ж': 'с',
 'з': 'б',
 'и': 'в',
 'й': ' ',
 'к': 'ё',
 'л': 'г',
 'м': 'ю',
 'н': 'щ',
 'о': 'и',
 'п': 'д',
 'р': 'э',
 'с': 'й',
 'т': 'ь',
 'у': 'м',
 'ф': 'р',
 'х': 'о',
 'ц': 'х',
 'ч': 'ф',
 'ш': 'к',
 'щ': 'ж',
 'ъ': 'ъ',
 'ы': 'ы',
 'ь': 'а',
 'э': 'ц',
 'ю': 'е',
 'я': 'л',
 ' ': 'з'}

In [12]:
def make_random_replacements(text):
    random_mapping = get_random_mapping()
    new_text = ''
    for symbol in text:
        new_text += random_mapping.get(symbol, 'ё')
    return new_text

In [13]:
make_random_replacements('аббввв')

'яннррр'

In [14]:
def get_mapping_by_frequencies(frequencies, text_frequencies, n=1):
    if n == 1:
        mapping = dict()
        frequencies_copy = frequencies.copy()
        for symbol, frequency in text_frequencies.items():
            closest_symbol = min(frequencies.items(), key=lambda x: abs(frequency - x[1]))[0]
            mapping[symbol] = closest_symbol
            if closest_symbol in frequencies_copy.keys():
                del frequencies_copy[closest_symbol]
        return mapping
    text_frequencies = sorted(text_frequencies.items(), key=lambda x: x[1], reverse=True)
    frequencies_sorted = sorted(frequencies.items(), key=lambda x: x[1], reverse=True)
    mapping = dict()
    for item in text_frequencies:
        filtered_frequencies = []
        for i in range(n):
            if item[0][i] in mapping.keys():
                for ngram, frequency in frequencies_sorted:
                    if ngram[i] == mapping[item[0][i]]:
                        filtered_frequencies.append((ngram, frequency))
        if len(filtered_frequencies) == 0:
            filtered_frequencies = frequencies_sorted.copy()
        closest_ngram = min(filtered_frequencies, key=lambda x: abs(item[1] - x[1]))[0]
        for i in range(n):
            if item[0][i] not in mapping.keys():
                mapping[item[0][i]] = closest_ngram[i]
    return mapping

In [15]:
test_frequencies_in_text = {
    'г': 0.2,
    'д': 0.3,
    'е': 0.5
}
get_mapping_by_frequencies(test_frequencies, test_frequencies_in_text)

{'г': 'а', 'д': 'б', 'е': 'в'}

In [16]:
def make_replacements_based_on_frequencies(text, frequencies, n=1):
    text_frequencies = get_ngram_frequencies(text, n)
    mapping = get_mapping_by_frequencies(frequencies, text_frequencies, n)
    new_text = ''
    for symbol in text:
        new_text += mapping.get(symbol, 'ё')
    return new_text
    

In [17]:
test_frequencies

{'а': 0.16666666666666666, 'б': 0.3333333333333333, 'в': 0.5}

In [18]:
make_replacements_based_on_frequencies('гддеее', test_frequencies, n=1)

'аббввв'

In [19]:
test_frequencies = {
    'аб': 0.2,
    'бб': 0.2,
    'бв': 0.2,
    'вв': 0.4
}

test_frequencies_in_text = {
    'гд': 0.2,
    'дд': 0.2,
    'де': 0.2,
    'ее': 0.4
}
mapping = get_mapping_by_frequencies(test_frequencies, test_frequencies_in_text, n=2)

In [20]:
mapping

{'е': 'в', 'г': 'а', 'д': 'б'}

In [21]:
make_replacements_based_on_frequencies('гддеее', test_frequencies, n=2)

'аббввв'

### 1. Базовый частотный метод по Шерлоку Холмсу

In [22]:
text = """
    Я помню чудное мгновенье:\n
    Передо мной явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    В томленьях грусти безнадежной,\n
    В тревогах шумной суеты,\n
    Звучал мне долго голос нежный\n
    И снились милые черты.\n

    Шли годы. Бурь порыв мятежный\n
    Рассеял прежние мечты,\n
    И я забыл твой голос нежный,\n
    Твои небесные черты.\n
    
    В глуши, во мраке заточенья\n
    Тянулись тихо дни мои\n
    Без божества, без вдохновенья,\n
    Без слез, без жизни, без любви.\n
"""

In [23]:
text = preprocess_text(text)

In [24]:
text

'я помню чудное мгновенье передо мной явилась ты как мимолетное виденье как гений чистой красоты в томленьях грусти безнадежной в тревогах шумной суеты звучал мне долго голос нежный и снились милые черты шли годы бурь порыв мятежный рассеял прежние мечты и я забыл твой голос нежный твои небесные черты в глуши во мраке заточенья тянулись тихо дни мои без божества без вдохновенья без слез без жизни без любви'

In [25]:
frequencies = get_ngram_frequencies(text, n=1)

In [26]:
frequencies

{'я': 0.022058823529411766,
 ' ': 0.1642156862745098,
 'п': 0.00980392156862745,
 'о': 0.07352941176470588,
 'м': 0.031862745098039214,
 'н': 0.06862745098039216,
 'ю': 0.004901960784313725,
 'ч': 0.01715686274509804,
 'у': 0.0196078431372549,
 'д': 0.0196078431372549,
 'е': 0.09558823529411764,
 'г': 0.022058823529411766,
 'в': 0.0392156862745098,
 'ь': 0.022058823529411766,
 'р': 0.02696078431372549,
 'й': 0.022058823529411766,
 'и': 0.05392156862745098,
 'л': 0.0392156862745098,
 'а': 0.029411764705882353,
 'с': 0.03676470588235294,
 'т': 0.04411764705882353,
 'ы': 0.03431372549019608,
 'к': 0.014705882352941176,
 'х': 0.00980392156862745,
 'б': 0.02696078431372549,
 'з': 0.02696078431372549,
 'ж': 0.01715686274509804,
 'ш': 0.007352941176470588}

In [27]:
text_encoded = make_random_replacements(text)

In [28]:
text_encoded

'цьирзхсьягкхрбьзъхробхэбьибшбкрьзхрвьцотчлеэьжпьфлфьзтзрчбжхрбьоткбхэбьфлфьъбхтвьятежрвьфшлержпьоьжрзчбхэцдьъшгежтьюбнхлкбйхрвьоьжшборълдьагзхрвьегбжпьногялчьзхбькрчърьърчреьхбйхпвьтьехтчтеэьзтчпбьябшжпьачтьъркпьюгшэьиршпоьзцжбйхпвьшлеебцчьишбйхтбьзбяжпьтьцьнлюпчьжорвьърчреьхбйхпвьжортьхбюбехпбьябшжпьоьъчгатьорьзшлфбьнлжрябхэцьжцхгчтеэьжтдрькхтьзртьюбньюрйбежольюбньокрдхробхэцьюбньечбньюбньйтнхтьюбньчсюот'

In [29]:
corpus_frequencies = get_ngram_frequencies(corpus, n=1)

In [30]:
text_decoded = make_replacements_based_on_frequencies(text_encoded, corpus_frequencies, n=1)

In [31]:
text_decoded

'у жераю ьппаео руаевоауо жокопе раеу увивкру ср бкб риревосаео випоауо бкб уоаиу ьирсеу бккреср в сервоаууж укпрси кокакпоьаеу в сковеукж шпраеу рпоср квпькв рао певуе уевер аоьару и раивиру ривро ьокср шви уепр кпку жекрв русоьару ккрроув жкоьаио роьср и у кккрв свеу уевер аоьару свеи аокораро ьокср в увпши ве рккбо кксеьоауу суапвиру сиже паи реи кок кеьорсвк кок впежаевоауу кок рвок кок ьикаи кок вюкви'

In [32]:
def get_accuracy(expected_text, recieved_text):
    matches_count = 0
    for i in range(len(expected_text)):
        if expected_text[i] == recieved_text[i]:
            matches_count += 1
    return matches_count / len(expected_text)

In [33]:
get_accuracy(text, text_decoded)

0.2696078431372549

Декодированный текст абсолютно нечитаемый, точность тоже оставляет желать лучшего.

### 2. Базовый подход с биграмммами

In [34]:
frequencies = get_ngram_frequencies(text, n=2)

In [35]:
frequencies

{'я ': 0.009828009828009828,
 ' п': 0.009828009828009828,
 'по': 0.004914004914004914,
 'ом': 0.004914004914004914,
 'мн': 0.009828009828009828,
 'ню': 0.002457002457002457,
 'ю ': 0.002457002457002457,
 ' ч': 0.009828009828009828,
 'чу': 0.002457002457002457,
 'уд': 0.002457002457002457,
 'дн': 0.004914004914004914,
 'но': 0.0171990171990172,
 'ое': 0.004914004914004914,
 'е ': 0.022113022113022112,
 ' м': 0.022113022113022112,
 'мг': 0.002457002457002457,
 'гн': 0.002457002457002457,
 'ов': 0.004914004914004914,
 'ве': 0.004914004914004914,
 'ен': 0.014742014742014743,
 'нь': 0.012285012285012284,
 'ье': 0.004914004914004914,
 'пе': 0.002457002457002457,
 'ер': 0.007371007371007371,
 'ре': 0.007371007371007371,
 'ед': 0.002457002457002457,
 'до': 0.007371007371007371,
 'о ': 0.009828009828009828,
 'ой': 0.012285012285012284,
 'й ': 0.022113022113022112,
 ' я': 0.004914004914004914,
 'яв': 0.002457002457002457,
 'ви': 0.007371007371007371,
 'ил': 0.007371007371007371,
 'ла': 0.0024570

In [36]:
text_encoded = make_random_replacements(text)

In [37]:
text_encoded

'отрмшяптсжчямётшуямыёякётрёьёчмтшямгтоыфлю ктйатеюетшфшмлёйямётыфчёякётеюетуёяфгтсф ймгтеью мйатытймшлёякодтуьж йфтнёэяючёцямгтытйьёымуюдтвжшямгт жёйатэыжсюлтшяётчмлумтумлм тяёцяагтфт яфлф ктшфлаётсёьйатвлфтумчатнжьктрмьаытшойёцяагтью  ёолтрьёцяфётшёсйатфтотэюналтйымгтумлм тяёцяагтйымфтяёнё яаётсёьйатытулжвфтымтшьюеётэюймсёякотйояжлф ктйфдмтчяфтшмфтнёэтнмцё йыютнёэтычмдямыёякотнёэт лёэтнёэтцфэяфтнёэтлпныф'

In [38]:
corpus_frequencies = get_ngram_frequencies(corpus, n=2)

In [39]:
text_decoded = make_replacements_based_on_frequencies(text_encoded, corpus_frequencies, n=2)

In [40]:
text_decoded

'ь к сей киле о све воено ковол  се о ьвокнин се тнт сос косе о волоено тнт воеоо коис о твни се в с скоеньк ввиисо со енлоне о в свов внк еисе о ииосе  викнк сео л кв  в к и еонеео о иеокоин сокео ковсе еко в ле сивн к вев сьсонеео внииоьк квонеоо соксе о ь  нсек св о в к и еонеео св о еосоиеео ковсе в вкиео в  свнто  нс коень сьеикоин сок  лео с о со  с ноисвн со  вл ке воень со  ико  со  но ео со  кйсво'

In [41]:
get_accuracy(text, text_decoded)

0.2034313725490196

In [42]:
corpus_frequencies

{'ан': 0.004731376333442366,
 'нн': 0.0028723919845321543,
 'на': 0.010689374076287717,
 'а ': 0.01803497390914182,
 ' к': 0.009553518369779473,
 'ка': 0.007605480878406496,
 'ар': 0.002420276030490426,
 'ре': 0.0054056970035330125,
 'ен': 0.007082721806545749,
 'ни': 0.007366578698145811,
 'ин': 0.003833138386870827,
 ' о': 0.013053991892738915,
 'од': 0.004235590088385244,
 'ди': 0.0022130562182213005,
 'н ': 0.005155235040356487,
 ' и': 0.010934698358168314,
 'из': 0.0017369644181319049,
 'з ': 0.001035670921237633,
 ' с': 0.015876719624743758,
 'са': 0.0016641805997729144,
 'ам': 0.002464374461613814,
 'мы': 0.0008344450704804246,
 'ых': 0.0008280229688605137,
 'х ': 0.002844134737404546,
 ' з': 0.004093019432423222,
 'зн': 0.0018093200963829011,
 'ме': 0.0028672543032362253,
 'ит': 0.003643472319029458,
 'ты': 0.0014809366335514565,
 ' р': 0.0039804185840207834,
 'ро': 0.00683611310434117,
 'ом': 0.004796453629857464,
 'ма': 0.0025328768788928637,
 'но': 0.00927265845893537,
 'ов'

С использованием биграмм качество упало. Возможно, причина в том, что биграмм сильно больше, чем униграмм. Поэтому в случае маленьких текстов сложнее подобрать правильные замены по частоте встречаемости. Проверим, будут ли биграммы повышать качество в случае текстов бОльших размеров:

In [43]:
text = """
    Я помню чудное мгновенье:\n
    Передо мной явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    В томленьях грусти безнадежной,\n
    В тревогах шумной суеты,\n
    Звучал мне долго голос нежный\n
    И снились милые черты.\n

    Шли годы. Бурь порыв мятежный\n
    Рассеял прежние мечты,\n
    И я забыл твой голос нежный,\n
    Твои небесные черты.\n
    
    В глуши, во мраке заточенья\n
    Тянулись тихо дни мои\n
    Без божества, без вдохновенья,\n
    Без слез, без жизни, без любви.\n
    
    Душе настало пробужденье:\n
    И вот опять явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    И сердце бьется в упоенье,\n
    И для него воскресли вновь\n
    И божество, и вдохновенье,\n
    И жизнь, и слезы, и любовь.\n
"""

In [44]:
text = preprocess_text(text)

In [45]:
text

'я помню чудное мгновенье передо мной явилась ты как мимолетное виденье как гений чистой красоты в томленьях грусти безнадежной в тревогах шумной суеты звучал мне долго голос нежный и снились милые черты шли годы бурь порыв мятежный рассеял прежние мечты и я забыл твой голос нежный твои небесные черты в глуши во мраке заточенья тянулись тихо дни мои без божества без вдохновенья без слез без жизни без любви душе настало пробужденье и вот опять явилась ты как мимолетное виденье как гений чистой красоты и сердце бьется в упоенье и для него воскресли вновь и божество и вдохновенье и жизнь и слезы и любовь'

##### Униграммы:

In [46]:
frequencies = get_ngram_frequencies(text, n=1)
text_encoded = make_random_replacements(text)

In [47]:
text_encoded

'ем ьчюъмбяфюьвмчщюьнвюгвм втвфьмчюьжменалсзгмэпмёсёмчачьлвэюьвмнафвюгвмёсёмщвюажмбазэьжмётсзьэпмнмэьчлвюгеймщтязэамрвхюсфвцюьжмнмэтвньщсймкячюьжмзявэпмхнябслмчювмфьлщьмщьльзмювцюпжмамзюалазгмчалпвмбвтэпмкламщьфпмрятгм ьтпнмчеэвцюпжмтсззвелм твцюавмчвбэпмамемхсрплмэньжмщьльзмювцюпжмэньамюврвзюпвмбвтэпмнмщлякамньмчтсёвмхсэьбвюгемэеюялазгмэайьмфюамчьамрвхмрьцвзэнсмрвхмнфьйюьнвюгемрвхмзлвхмрвхмцахюамрвхмлърнамфяквмюсзэсльм тьряцфвюгвмамньэмь еэгменалсзгмэпмёсёмчачьлвэюьвмнафвюгвмёсёмщвюажмбазэьжмётсзьэпмамзвтфивмргвэземнмя ьвюгвмамфлемювщьмньзётвзламнюьнгмамрьцвзэньмамнфьйюьнвюгвмамцахюгмамзлвхпмамлърьнг'

In [48]:
corpus_frequencies = get_ngram_frequencies(corpus, n=1)
text_decoded = make_replacements_based_on_frequencies(text_encoded, corpus_frequencies, n=1)

In [49]:
text_decoded

'п чедаю чяуаео дяаесоако чодоуе даея пснвклк ск пкп дндевосаео снуоако пкп яоаня чнлсея пдклеск с седвоакпй ядялсн допакуоьаея с сдосеякй хядаея ляоск псячкв дао уевяе яевел аоьакя н ланвнлк днвко чодск хвн яеук дядк чедкс дпсоьакя дкллопв чдоьано дочск н п пкдкв ссея яевел аоьакя ссен аодолако чодск с явяхн се ддкпо пксечоакп спаявнлк снйе уан ден доп деьолсск доп суейаесоакп доп лвоп доп ьнпан доп вюдсн уяхо аклскве чдедяьуоако н сес ечпск пснвклк ск пкп дндевосаео снуоако пкп яоаня чнлсея пдклеск н лодуфо дкослп с ячеоако н увп аояе селпдолвн саеск н деьолссе н суейаесоако н ьнпак н лвопк н вюдеск'

In [50]:
get_accuracy(text, text_decoded)

0.18616144975288304

##### Биграммы:

In [51]:
corpus_frequencies = get_ngram_frequencies(corpus, n=2)
text_decoded = make_replacements_based_on_frequencies(text_encoded, corpus_frequencies, n=2)

In [52]:
text_decoded

'ь тсс и тяо со си ссо со тотоос с са ьсот кс ое к к сосстоо со сооо со к к ио оа токоса кт ксое с оссто сью итякоо сон  оон са с отосси ю ляс са кяоое нсят т с о остис истск  он еа о к отокс сотео тотое лто исое сятс тстес сьоон еа т ккоьт ттон оо сотое о ь н сет осса истск  он еа оссо  осок ео тотое с итяло сс ст ко н осто сь оь ятокс ооюс о о ссо сон сснокос  сон сосю ссо сь сон ктон сон нон о сон тиссо ояло   ко тс ттссяноо со о ссо стьос ьсот кс ое к к сосстоо со сооо со к к ио оа токоса кт ксое о котоко ссоокь с ятсо со о оть  оис сскктокто с ссс о сснокосс о сосю ссо со о нон с о ктоне о тисссс'

In [53]:
get_accuracy(text, text_decoded)

0.18780889621087316

В случае текста большего размера биграммы дают чуть лучшую точнось, чем униграммы. Но текст по-прежнему нечитаемый. Возможно, МСМС-сэмплирование исправит ситуацию:

### 3. МСМС-сэмплирование
На частоту биграммы можно смотреть как на вероятность того, что за первым символом, входящим в неё, следует второй. То есть частота биграммы - вероятность перехода между состояниями Марковской цепи.

Будем производить МСМС-сэмрлирование следующим образом: будем начинать с перестановки, полученной путём применения частотного метода из пункта 2. Далее будем вычислять вероятность получения текущей перестановки $p$ как произведение частот биграмм, входящих в неё.

"Блуждание" будем осуществлять следующим образом: будем менять местами 2 символа текущей перестановки, считать вероятность перестановки $q$, и принимать новую перестановку с вероятностью $\frac{q}{p}$, где $p$ - вероятность текущей перестановки (под вероятностью перестановки понимается вероятность сгенерировать полученный путём применения перестановки текст).

In [54]:
def get_probability(text, frequencies):
    probability = 0.0
    for idx in range(len(text) - 1):
        bigram = text[idx:idx + 2]
        if bigram in frequencies:
            probability += np.log(frequencies[bigram])
        else:
            probability += np.log(1.0 / (len(RUSSIAN_SYMBOLS) ** 2 + len(text)))
    return probability

In [55]:
def make_replacements(text, frequencies, mapping):
    new_text = ''
    for symbol in text:
        new_text += mapping.get(symbol, 'ь')
    return new_text

In [56]:
def get_bigram_frequencies(text):
    text = ["".join(ngram) for ngram in nltk.everygrams(text, min_len=2, max_len=2)]
    frequenciess = {
        k: (v + 1) / (len(text) + len(set(text)) ** 2) for k, v in Counter(text).items()
    }
    return frequenciess

In [64]:
def mcmc(text_encoded, frequencies, symbols_list, iterations):
    corpus_symbols = RUSSIAN_SYMBOLS.copy()
    symbols_encoded = symbols_list.copy()
    np.random.shuffle(corpus_symbols)
    np.random.shuffle(symbols_encoded)
    mapping = {i: j for i, j in zip(symbols_encoded, corpus_symbols[:len(symbols_encoded)])}
    text_decoded = make_replacements(text_encoded, frequencies, mapping)
    best_probability = get_probability(text_decoded, frequencies)
    best_decoded_text = text_decoded
    current_probability = best_probability
    for _ in range(iterations):
        i, j = np.random.choice(len(corpus_symbols), size=2, replace=False)
        new_symbols = corpus_symbols.copy()
        new_symbols[i], new_symbols[j] = new_symbols[j], new_symbols[i]
        new_mapping = {i: j for i, j in zip(symbols_encoded, new_symbols[:len(symbols_encoded)])}
        text_decoded = make_replacements(text_encoded, frequencies, new_mapping)
        probability = get_probability(text_decoded, frequencies)
        if np.random.rand() < np.exp(probability - current_probability):
            current_probability = probability
            best_probability = probability
            best_decoded_text = text_decoded
            corpus_symbols = new_symbols
            mapping = new_mapping
    return best_decoded_text

In [65]:
text = """
    Я помню чудное мгновенье:\n
    Передо мной явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    В томленьях грусти безнадежной,\n
    В тревогах шумной суеты,\n
    Звучал мне долго голос нежный\n
    И снились милые черты.\n

    Шли годы. Бурь порыв мятежный\n
    Рассеял прежние мечты,\n
    И я забыл твой голос нежный,\n
    Твои небесные черты.\n
    
    В глуши, во мраке заточенья\n
    Тянулись тихо дни мои\n
    Без божества, без вдохновенья,\n
    Без слез, без жизни, без любви.\n
    
    Душе настало пробужденье:\n
    И вот опять явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    И сердце бьется в упоенье,\n
    И для него воскресли вновь\n
    И божество, и вдохновенье,\n
    И жизнь, и слезы, и любовь.\n
"""
text = preprocess_text(text)

In [66]:
text

'я помню чудное мгновенье передо мной явилась ты как мимолетное виденье как гений чистой красоты в томленьях грусти безнадежной в тревогах шумной суеты звучал мне долго голос нежный и снились милые черты шли годы бурь порыв мятежный рассеял прежние мечты и я забыл твой голос нежный твои небесные черты в глуши во мраке заточенья тянулись тихо дни мои без божества без вдохновенья без слез без жизни без любви душе настало пробужденье и вот опять явилась ты как мимолетное виденье как гений чистой красоты и сердце бьется в упоенье и для него воскресли вновь и божество и вдохновенье и жизнь и слезы и любовь'

In [67]:
corpus_frequencies = get_bigram_frequencies(corpus)
text_encoded = make_random_replacements(text)

In [68]:
text_encoded

'ляу дймяеёъй ьядхй вьйшьяуьэьъ ядй чялвтныишяпбяоыоядтд ньпй ьявтъьйшьяоыояхьйтчяетип чяоэыи пбявяп дньйшлфяхэёиптяаькйыъьжй чявяпэьв хыфязёдй чяиёьпбяквёеынядйьяъ нх ях н ияйьжйбчятяийтнтишядтнбьяеьэпбязнтях ъбяаёэшяу эбвядлпьжйбчяэыииьлняуэьжйтьядьепбятялякыабняпв чях н ияйьжйбчяпв тяйьаьийбьяеьэпбявяхнёзтяв ядэыоьякып еьйшляплйёнтишяптф яъйтяд тяаькяа жьипвыяаькявъ фй вьйшляаькяинькяаькяжткйтяаькянмавтяъёзьяйыипын яуэ аёжъьйшьятяв пя улпшялвтныишяпбяоыоядтд ньпй ьявтъьйшьяоыояхьйтчяетип чяоэыи пбятяиьэъщьяашьпилявяёу ьйшьятяънляйьх яв иоэьинтявй вшятяа жьипв ятявъ фй вьйшьятяжткйшятяинькбятянма вш'

In [69]:
text_decoded = mcmc(text_encoded, corpus_frequencies, RUSSIAN_SYMBOLS, 10000)

In [70]:
text_decoded

'я гомню чудное мпновенье гередо мной явилась ты как мимолетное виденье как пений чистой красоты в томленьяз прусти бехнадежной в тревопаз шумной суеты хвучал мне долпо полос нежный и снились милые черты шли поды бурь горыв мятежный рассеял грежние мечты и я хабыл твой полос нежный твои небесные черты в плуши во мраке хаточенья тянулись тизо дни мои бех божества бех вдозновенья бех слех бех жихни бех любви душе настало гробужденье и вот огять явилась ты как мимолетное виденье как пений чистой красоты и сердёе бьется в угоенье и для непо воскресли вновь и божество и вдозновенье и жихнь и слехы и любовь'

In [74]:
text_decoded = mcmc(text_encoded, corpus_frequencies, RUSSIAN_SYMBOLS, 10000)

In [75]:
text_decoded

'я гокню чудное кпновенае гередо кной явилыса ть хых киколетное виденае хых пений чистой хрысоть в токленаяз прусти бемныдежной в тревопыз шукной суеть мвучыл кне долпо полос нежньй и снилиса килье черть шли подь бура горьв кятежньй рыссеял грежние кечть и я мыбьл твой полос нежньй твои небеснье черть в плуши во крыхе мыточеная тянулиса тизо дни кои бем божествы бем вдозновеная бем слем бем жимни бем любви душе ныстыло гробужденае и вот огята явилыса ть хых киколетное виденае хых пений чистой хрысоть и сердёе бается в угоенае и для непо восхресли внова и божество и вдозновенае и жимна и слемь и любова'

In [76]:
get_accuracy(text, text_decoded)

0.8056013179571664

Как мы видим не каждая попытка даёт хороший результат. Возможно, это связано с тем, что мы в процессе блужданий попадаем в окрестность плохих локальных максимумов и застреваем там. Попробуем исправить это следующим образом: будем совершать много запусков алгоритма, каждый раз стартуя с разных перестановок и выводить результат применения лучшей перестановки.

In [79]:
def mcmc(text_encoded, frequencies, symbols_list, iterations, attempts):
    best_probability = -np.inf
    best_decoded_text = ''
    for attampt in range(attempts):
        corpus_symbols = RUSSIAN_SYMBOLS.copy()
        symbols_encoded = symbols_list.copy()
        np.random.shuffle(corpus_symbols)
        np.random.shuffle(symbols_encoded)
        mapping = {i: j for i, j in zip(symbols_encoded, corpus_symbols[:len(symbols_encoded)])}
        text_decoded = make_replacements(text_encoded, frequencies, mapping)
        current_probability = get_probability(text_decoded, frequencies)
        current_decoded_text = text_decoded
        for _ in range(iterations):
            i, j = np.random.choice(len(corpus_symbols), size=2, replace=False)
            new_symbols = corpus_symbols.copy()
            new_symbols[i], new_symbols[j] = new_symbols[j], new_symbols[i]
            new_mapping = {i: j for i, j in zip(symbols_encoded, new_symbols[:len(symbols_encoded)])}
            text_decoded = make_replacements(text_encoded, frequencies, new_mapping)
            probability = get_probability(text_decoded, frequencies)
            if np.random.rand() < np.exp(probability - current_probability):
                current_probability = probability
                current_decoded_text = text_decoded
                corpus_symbols = new_symbols
                mapping = new_mapping
        if current_probability > best_probability:
            best_probability = current_probability
            best_decoded_text = current_decoded_text
    return best_decoded_text

In [80]:
text_decoded = mcmc(text_encoded, corpus_frequencies, RUSSIAN_SYMBOLS, 10000, 100)

In [81]:
text_decoded

'я гомню чудное мпновенье гередо мной явилась ты как мимолетное виденье как пений чистой красоты в томленьях прусти безнадежной в тревопах шумной суеты звучал мне долпо полос нежный и снились милые черты шли поды бурь горыв мятежный рассеял грежние мечты и я забыл твой полос нежный твои небесные черты в плуши во мраке заточенья тянулись тихо дни мои без божества без вдохновенья без слез без жизни без любви душе настало гробужденье и вот огять явилась ты как мимолетное виденье как пений чистой красоты и сердёе бьется в угоенье и для непо воскресли вновь и божество и вдохновенье и жизнь и слезы и любовь'

In [82]:
get_accuracy(text, text_decoded)

0.9686985172981878

МСМС-сэмплирование даёт очень хорошую точность относительно рассмотренных ранее методов. Текст декодируется достаточно хорошо (можно разобрать слова и понять смысл).

### 4. Расшифруем сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

In [83]:
message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [84]:
message_decoded = mcmc(message, corpus_frequencies, list(set(message)), 10000, 100)

In [85]:
message_decoded

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю'

### 5. Триграммы и четыреграммы

In [98]:
def get_ngram_frequencies(text, n=2):
    text = ["".join(ngram) for ngram in nltk.everygrams(text, min_len=n, max_len=n)]
    frequenciess = {
        k: (v + 1) / (len(text) + len(set(text)) ** n) for k, v in Counter(text).items()
    }
    return frequenciess

def get_average_accuracy(text_original, text_encoded, n, attempts=10,
                         mcmc_iterations=5000, mcmc_attempts=10):
    corpus_frequencies = get_ngram_frequencies(corpus, n)
    iterations = 10
    accuracies_sum = 0.0
    for _ in range(attempts):
        text_decoded = mcmc(text_encoded, corpus_frequencies, RUSSIAN_SYMBOLS, mcmc_iterations, mcmc_attempts)
        accuracies_sum += get_accuracy(text_original, text_decoded)
    return accuracies_sum / iterations

def make_experiments(text, N, attempts=10, mcmc_iterations=5000, mcmc_attempts=10):
    text = preprocess_text(text)
    text_encoded = make_random_replacements(text)
    print(f'Text length: {len(text)}')
    for n in N:
        accuracy = get_average_accuracy(text, text_encoded, n, attempts, mcmc_iterations, mcmc_attempts)
        print(f'{n}-gram accuracy: {accuracy}')
    

In [88]:
text = """
    Я помню чудное мгновенье:\n
    Передо мной явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    В томленьях грусти безнадежной,\n
    В тревогах шумной суеты,\n
    Звучал мне долго голос нежный\n
    И снились милые черты.\n

    Шли годы. Бурь порыв мятежный\n
    Рассеял прежние мечты,\n
    И я забыл твой голос нежный,\n
    Твои небесные черты.\n
    
    В глуши, во мраке заточенья\n
    Тянулись тихо дни мои\n
    Без божества, без вдохновенья,\n
    Без слез, без жизни, без любви.\n
    
    Душе настало пробужденье:\n
    И вот опять явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n

    И сердце бьется в упоенье,\n
    И для него воскресли вновь\n
    И божество, и вдохновенье,\n
    И жизнь, и слезы, и любовь.\n
"""

In [91]:
make_experiments(text, [2, 3, 4])

Text length: 607
2-gram accuracy: 0.8428336079077431
3-gram accuracy: 0.016803953871499175
4-gram accuracy: 0.023558484349258647


Триграммы и четыреграммы не улучшили качество декодирования. Возможно, это следствие того, что по коротким текстам не получается посчитать достаточно хорошую статистику для триграмм и четыреграмм. Попробуем взять текст подлиннее:

In [104]:
text = ''
with open('Borodino.txt', 'r', encoding='utf-8') as f:
    for line in f:
        text += ' ' + line

In [105]:
text[:100]

' — Скажи-ка, дядя, ведь не даром\n Москва, спаленная пожаром,\n Французу отдана?\n Ведь были ж схватки '

In [107]:
make_experiments(text, [2, 3], 10, 10000, 10)

Text length: 2378


C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_4044/3450065470.py:20: RuntimeWarning: overflow encountered in exp
  if np.random.rand() < np.exp(probability - current_probability):


2-gram accuracy: 1.0
3-gram accuracy: 0.018797308662741798


Качество декодирования на основе статистики триграмм по-прежнему оставляет желать лучшего. Возможно, для получения лучшего результата необходимо брать текст большей длины, но в связи с отсутствием достаточных мощностей для обработки текстов бОльших размеров за конечное время такие эксперименты не были проведены. Попробуем наоборот взять текст меньших размеров.

In [109]:
text = """
    Я помню чудное мгновенье:\n
    Передо мной явилась ты,\n
    Как мимолетное виденье,\n
    Как гений чистой красоты.\n
"""
make_experiments(text, [2, 3, 4], 10, 10000, 10)

Text length: 95
2-gram accuracy: 0.4094736842105263
3-gram accuracy: 0.033684210526315796
4-gram accuracy: 0.03368421052631579


По-прежнему метрики для триграмм и четыреграмм получились хуже, чем для биграмм. Но можно заметить, что точнось для биграмм понизилась относительно текстов бОльшей длины, для триграмм и четыреграмм точность наоборот стала выше.